In [1]:
import telepot
from telepot.loop import MessageLoop
from telepot.namedtuple import InlineKeyboardMarkup, InlineKeyboardButton
from telepot.delegate import (per_chat_id, per_application, call, create_open, pave_event_space)
from pprint import pprint
import sys
import time
import response_manager
import pandas as pd
%run Result_Word2Vec.ipynb



In [2]:
# Simulate a database to store unread messages
class MessageStore(object):
    def __init__(self):
        self._db = {}
        self._results_db = []
        self.tag = []
     

    def put_tag(self, t):
        if len(self.tag):
            del self.tag
            self.tag.append(t)
        else:
            self.tag.append(t)
    
    def pull_tag(self):
        t = self.tag
        return t 
    
    def put_results(self, results):
        if len(self._results_db):
            del self._results_db[0]
        self._results_db.append(results)
    
    
            
    def pull_results(self):
        if len(self._results_db):
            res = self._results_db
            return res
        else:
            return []

    def put(self, msg):
        chat_id = msg['chat']['id']

        if chat_id not in self._db:
            self._db[chat_id] = []

        self._db[chat_id].append(msg)

    # Pull all unread messages of a `chat_id`
    def pull(self, chat_id):
        
        if len(self._db):
            messages = self._db[chat_id]
            # sort by date
            messages.sort(key=lambda m: m['date'])
            return messages
        else:
            return []
            

    
    def delete_mess(self, chat_id):
        if len(self._db):
            del self._db[chat_id]
        # Tells how many unread messages per chat_id
        #return [(k,len(v)) for k,v in self._db.items()]

In [3]:
# Accept commands from owner. Give him unread messages.
class MessageHandler(telepot.helper.ChatHandler):
    def __init__(self, seed_tuple, store, **kwargs):
        super(MessageHandler, self).__init__(seed_tuple, **kwargs)
        self._store = store

    def _read_messages(self, messages):
        for msg in messages:
            # assume all messages are text
            self.sender.sendMessage(msg['text'])

    def on_chat_message(self, msg):
        #Flag per non entrare nella iterazione del yes or no quando si sta processando per la prima volta il problema 
        flag= True
        
        content_type, chat_type, chat_id = telepot.glance(msg)
        
        if content_type == 'text' :
            name = msg["from"]["first_name"]
            txt = str(msg['text'])
            
            
        else:
            self.sender.sendMessage("I don't understand")
            return
        
        #Con questo metodo creiamo una lista con i messaggi precedenti relativi alla chat con l'utente corrente 
        message_history = self._store.pull(chat_id)
        
        
        #Se il comando è start mando il messaggio di inzio ed elimino i messaggi precedenti salvati relativi all'utente 
        if '/start' == txt :
            #print(message_history)
            self.sender.sendMessage('Hello {}! I\'m StackBot. Tell me what is your problem involving Python language, try to be as general as possible'.format(name))
            self._store.delete_mess(chat_id)
            self._store.put(msg)
            
        elif len(message_history) == 0 and txt != '/start':
            self.sender.sendMessage('You have to write /start to begin the interaction')
            
        #Se l'unico messaggio salvato è il comando start procediamo con il primo passo del processo
        if len(message_history) == 1 and message_history[0]['text'] == '/start' :
            print(msg)
            #controlliamo che il messaggio appena inviato non sia un altro comando 
            if 'entities' in msg:
                self._store.delete_mess(chat_id)
                self.sender.sendMessage("You can't type this in your request, please restart your interaction")
                
            #altrimenti sarà il testo del problema da mandare al w2v
            else:
                # resp = response_manager.gen_response(txt)
                #Chiamo la funzione che mi rende la lista dei tags delle domande con maggiore similarità
                results = w2v_result(txt, None)
                #print(results)
                
                #Salviamo nel nostro "db" i risultati del w2v e il messaggio relativo al problema dell'utente
                self._store.put_results(results)
                self._store.put(msg)
                
                #dai risultati prelevo tutte le liste dei tag delle varie domande 
                tags = [el[2] for el in results]
                
                #ciclo su ogni lista di tag, e mi fermo quando trovo il primo tag
                if len(tags) > 0:
                    for tag in tags:
                        if len(tag) > 0:
                            t = tag[0]
                            break
                
                #Salvo il tag appena trovato 
                self._store.put_tag(t)
                
                #Chiedo all'utente se il suo problema ha a che fare con il tag appena estratto 
                #Notare che precedentemente sono stati eliminati i tag corrispondenti a parole già scritte dall'utente 
                #nella sua descrizione del problema
            
                self.sender.sendMessage("Your question has something to do with:" + t + "?")
                self.sender.sendMessage("Write Y for yes or N for no")
                
                flag = False
        
        #Analizzo il caso in cui sono presenti i messaggi relativi a /start e alla descrizione del problema
        #Quindi qui vado a scremare i risultati in base all'input dell'utente e al fatto se il tag a lui proposto 
        #centri o meno col suo problema 
        
        if len(message_history) > 1 and flag:
            res = []
            #Caso affermativo 
            if txt == 'Y' or txt == 'y':
                res = filter_results(self._store,1)
                print(res) 
                self._store.put_results(res)
                
            elif txt == 'N' or txt == 'n':
                res = filter_results(self._store,2)
                print(res) 
                self._store.put_results(res)
                
            self._store.put(msg)
            
                        
                        
    
        

In [4]:
def filter_results(store, choice):
        
        #Carico in variabili i precedenti risultati e il tag relativo 
        tmp_results = store.pull_results()
        results = tmp_results[0]
        tmp_tag_target = store.pull_tag()
        tag_target= tmp_tag_target[0] 
        
        print('tag target' + tag_target)
        tags = []
        
        print(results)
        #dai risultati prelevo tutte le liste dei tag delle varie domande
        #for el in results:
        #    print('element of results')
        #    print(el)
        #    tags.append(l[2])
        #    print('list of tags:')
        #    print(tags)
        tags = [el[2] for el in results]
            
        print(tags)
        
        i = 0
                
        if len(tags) > 0:
            for tag in tags:
                #choice == 1 vuol dire che l'utente è interessato al tag
                if (choice) == 1:
                    #Se il tag target non è presente nella lista dei tag relativi a questo risultato
                    if tag_target not in tag:
                        print('Removing the result')
                        #rimuovi la riga relativa dai risultati 
                        results.pop(i)
                elif choice == 2:
                    #Se il tag target è presente nella lista dei tag rimuovi la lista relativa a quel tag dai risultati 
                    if tag_target in tag:
                        print('Removing the result')
                        results.pop(i)
                i = i+1 
        
        return results

In [5]:
class ChatBox(telepot.DelegatorBot):
    def __init__(self, token, owner_id):
        self._owner_id = owner_id
        self._seen = set()
        self._store = MessageStore()
        
        super(ChatBox, self).__init__(token, [
            pave_event_space()(
                per_chat_id(), create_open, MessageHandler, self._store, timeout=20),
        ])

In [ ]:
OWNER_ID = 102199992
TOKEN = '2067569208:AAHYKIRPZJ_eKtEFAtAmcFOWGlOMyRbGsXI'

bot = ChatBox(TOKEN, OWNER_ID)
MessageLoop(bot).run_as_thread()
print('Listening ...')

while 1:
    time.sleep(3)

Listening ...
{'message_id': 368, 'from': {'id': 102199992, 'is_bot': False, 'first_name': 'Marco', 'last_name': 'Lilliu', 'username': 'Marco_Lilliu', 'language_code': 'it'}, 'chat': {'id': 102199992, 'first_name': 'Marco', 'last_name': 'Lilliu', 'username': 'Marco_Lilliu', 'type': 'private'}, 'date': 1638466982, 'text': 'Problem loading csv file'}
(44325, 0.8386609800062548, ['html', 'flask'])
(17538, 0.8336420897111777, [])
(16133, 0.8316919039713923, [])
(71641, 0.8312103703959228, ['pandas', 'dataframe', 'jupyter-notebook', 'ipywidgets'])
(40184, 0.8222910639811026, ['windows', 'csv'])
(7270, 0.82100621317191, ['pandas'])
(39442, 0.8178355091768633, ['arrays', 'numpy', 'scipy', 'mat-file'])
(63430, 0.8139666617397625, [])
(36160, 0.8116483410383097, ['csv'])
(52680, 0.8110004009722135, ['pandas'])
tag targethtml
[(44325, 0.8386609800062548, ['html', 'flask']), (17538, 0.8336420897111777, []), (16133, 0.8316919039713923, []), (71641, 0.8312103703959228, ['pandas', 'dataframe', 'jupy

ERROR:root:on_close() called due to IdleTerminate: 20
